In [15]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", use_fast=False)
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=False)

from sentence_transformers import SentenceTransformer, util
bi_encoder = SentenceTransformer('nq-distilbert-base-v1')

from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])
import src.clean_dataset as clean

import pandas as pd
import time
import pickle5 as pickle

In [16]:
#import data:
df = pd.read_csv('../data/processed/taxonomy_final.csv')

corpus_embeddings = pickle.load(open("../data/processed/splitted_corpus_embeddings.pkl", 'rb'))

In [37]:
wrapped, splitted = clean.split_at_length(df, 'all_text_clean', 512)
passages = splitted.text.tolist()
passage_id = splitted.PIMS_ID.tolist()
#corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [11]:
#pickle.dump(corpus_embeddings, open("../data/processed/splitted_corpus_embeddings.pkl", "wb" ))

In [ ]:
splitted = clean.split_at_length(df, 'all_text', 1000)
passages_long = splitted.text.tolist()
print(len(passages_long))
#corpus_embeddings_long = bi_encoder.encode(passages_long, convert_to_tensor=True, show_progress_bar=True)

In [24]:
query = input()

top_k = 5  # Number of passages we want to retrieve with the bi-encoder
start_time = time.time()
question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
hits = hits[0]  # Get the hits for the first query
end_time = time.time()

how to mitigate floods?


In [63]:
# Output of top-k hits
print("Input question:", query)
print("Results (after {:.3f} seconds):".format(end_time - start_time))
matches = []
ids = []
scores =  []
for hit in hits:
    print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))
    matches.append(passages[hit['corpus_id']])
    ids.append(passage_id[hit['corpus_id']])
    scores.append(hit['score'])
    
print("\n\n========\n")
#d = {'PIMS_ID':ids,'context':matches}
#df = pd.DataFrame(d)
scores

Input question: how to mitigate floods?
Results (after 0.906 seconds):
	0.501	coastline with climate resilient shoreline and flood protection measures introduced including vegetation planting along the coast and riparian streams and beach replenishment by the end of the programme at least inhabitants in villages have their water supply and associated infrastructure improved to manage climateinduced impacts on water supply the target villages lack robust water supply system to withstand climateinduced impacts in water supply n of population and communities accessing improved water
	0.443	implementing water adaptive productive practices
	0.438	enabling climate vulnerable people to plan for and adapt to the impacts of climate change knowledge will be generated and used to formulate comprehensive communitybased adaptation plans ecological and physical infrastructure measures for water management will be adopted to regulate baseflow and reduce risk of floods while mitigating against drought

[0.50050205, 0.44299835, 0.43815923, 0.43215483, 0.40131235]

In [55]:
answers = []
for match in matches:
    inputs = tokenizer.encode_plus(query, match, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {query}")
    print(f"Answer: {answer}\n")
    answers.append(answer)
    
#df = df.assign(answer=answers)

Question: how to mitigate floods?
Answer: vegetation planting along the coast and riparian streams and beach replenishment

Question: how to mitigate floods?
Answer: implementing water adaptive productive practices

Question: how to mitigate floods?
Answer: ecological and physical infrastructure measures for water management

Question: how to mitigate floods?
Answer: vegetation planting

Question: how to mitigate floods?
Answer: climateresilient river basin flood risk management approach



In [66]:
df_results = pd.DataFrame(
    {'PIMS_ID': ids,
     'answer': answers,
     'context': matches,
     "scores": scores
    })
df_results.set_index('PIMS_ID', inplace=True)

In [67]:
df_results

,answer,context,scores
PIMS_ID,,,
4667,vegetation planting along the coast and ripari...,coastline with climate resilient shoreline and...,0.500502
5757,implementing water adaptive productive practices,implementing water adaptive productive practices,0.442998
4508,ecological and physical infrastructure measure...,enabling climate vulnerable people to plan for...,0.438159
4667,vegetation planting,change and variabilityinduced stress by the co...,0.432155
6215,climateresilient river basin flood risk manage...,the subregion in line with the drin mou implem...,0.401312
